In [1]:
from transformers import pipeline
import pandas as pd
import sklearn
import transformers
import nltk
import spacy

c:\Users\Manuel\anaconda3\envs\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
data = pd.read_csv('job_postings.csv')
data.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [10]:
desc = data['description']
frau = data['fraudulent']

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
desc_bow = vectorizer.fit_transform(desc)


In [9]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer() 
docs_tfidf = tfidf.fit_transform(desc_bow)
docs_tfidf_densa = docs_tfidf.todense()
docs_tfidf_densa

matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.07446071, 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]])

In [12]:
from sklearn.model_selection import train_test_split
desc_train, desc_test, frau_train, frau_test = train_test_split(docs_tfidf, frau, test_size = 0.25, 
                                                                    random_state = 50)

In [13]:
# Entrenamiento del clasificador NB

from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(desc_train, frau_train)

MultinomialNB()

In [15]:

frau_pred = clf.predict(desc_test)

In [16]:
#confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(frau_test, frau_pred)
cm

array([[485,   0],
       [180,  52]], dtype=int64)

In [17]:
acc_train = clf.score(desc_train, frau_train)
acc_test = clf.score(desc_test, frau_test)

print("Accuracy entrenamiento: ", acc_train)
print("Accuracy PRUEBA: ", acc_test)
print("Fiabilidad: ", acc_test / acc_train)  

Accuracy entrenamiento:  0.7774674115456238
Accuracy PRUEBA:  0.7489539748953975
Fiabilidad:  0.9633252323804274


In [18]:
# Entrenamiento del clasificador SVM

from sklearn.svm import SVC
svm = SVC(kernel='linear')
# svm = SVC(kernel='poly')
# svm = SVC(kernel='rbf')
# svm = SVC(kernel='sigmoid')


svm.fit(desc_train, frau_train)

acc_train = svm.score(desc_train, frau_train)
acc_test = svm.score(desc_test, frau_test)

print("Accuracy entrenamiento: ", acc_train)
print("Accuracy PRUEBA: ", acc_test)
print("Fiabilidad: ", acc_test / acc_train)

Accuracy entrenamiento:  0.9823091247672253
Accuracy PRUEBA:  0.8884239888423989
Fiabilidad:  0.9044240417220251
